<a href="https://colab.research.google.com/github/lucasanl/Data-School/blob/main/school.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://bit.ly/3ZLyF82 -O CSS.css -q

from IPython.core.display import HTML
with open('./CSS.css', 'r') as file:
    custom_css = file.read()

HTML(custom_css)


In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=a08757e948da4a9c8ebeb33c64da19e687e6a52b2c4c01069cc44e18920dc51a
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [26]:
# Import necessary libraries

import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
import sys
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
spark = SparkSession.builder.appName("SCHOOL-DATA").getOrCreate()

In [6]:
# Load the dataset
df = spark.read.csv("schoolinfo.csv", header=True, inferSchema=True)
# Print the memory usage of the dataframe
memory_in_bytes = df.rdd.flatMap(lambda x: x).map(lambda x: sys.getsizeof(x)).sum()
print(f"Memory usage: {memory_in_bytes/1024**2:.2f} MB")

Memory usage: 0.01 MB


In [8]:
# Checking the number of rows and columns in the dataset
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

Number of rows: 48
Number of columns: 8


In [9]:
# Displaying a few sample rows to understand the structure of the data
df.show(20)

+-----------+------+----+-----+----------+-----------+----+-----+
|     Region|Status|CTSA| GOVT|GOVT AIDED|INDEPENDENT| JNV|   KV|
+-----------+------+----+-----+----------+-----------+----+-----+
|      Ajmer| Regd.|   0| 8537|         0|     102253|4945| 9194|
|      Ajmer| Appd.|   0| 8509|         0|     101822|4928| 9160|
|      Ajmer|  Pass|   0| 8154|         0|     100178|4911| 8875|
|  Bengaluru| Regd.|  99|  151|         0|      60667|2442| 5108|
|  Bengaluru| Appd.|  97|  151|         0|      60555|2440| 5093|
|  Bengaluru|  Pass|  94|  151|         0|      60089|2440| 5042|
|     Bhopal| Regd.|   0| 5802|         0|      84877|4171| 9126|
|     Bhopal| Appd.|   0| 5681|         0|      84390|4160| 9094|
|     Bhopal|  Pass|   0| 4837|         0|      79048|4152| 8406|
|Bhubaneswar| Regd.| 102|17252|         0|      81012|5295|13489|
|Bhubaneswar| Appd.| 101|17036|         0|      80621|5277|13441|
|Bhubaneswar|  Pass|  96|15828|         0|      78070|5272|13136|
| Chandiga

In [10]:
# Print the column names and their data types
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- CTSA: integer (nullable = true)
 |-- GOVT: integer (nullable = true)
 |-- GOVT AIDED: integer (nullable = true)
 |-- INDEPENDENT: integer (nullable = true)
 |-- JNV: integer (nullable = true)
 |-- KV: integer (nullable = true)



In [11]:
# Checking for missing values
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------+------+----+----+----------+-----------+---+---+
|Region|Status|CTSA|GOVT|GOVT AIDED|INDEPENDENT|JNV| KV|
+------+------+----+----+----------+-----------+---+---+
|     0|     0|   0|   0|         0|          0|  0|  0|
+------+------+----+----+----------+-----------+---+---+



In [12]:
# Identifying the data types of the columns
df.dtypes

[('Region', 'string'),
 ('Status', 'string'),
 ('CTSA', 'int'),
 ('GOVT', 'int'),
 ('GOVT AIDED', 'int'),
 ('INDEPENDENT', 'int'),
 ('JNV', 'int'),
 ('KV', 'int')]

In [13]:
# Describing the statistical summary of the dataset
df.describe().show()

+-------+----------+------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|summary|    Region|Status|             CTSA|              GOVT|       GOVT AIDED|       INDEPENDENT|               JNV|                KV|
+-------+----------+------+-----------------+------------------+-----------------+------------------+------------------+------------------+
|  count|        48|    48|               48|                48|               48|                48|                48|                48|
|   mean|      null|  null|          30.6875|         17316.625|         1116.875|100303.52083333333|2846.9791666666665|         7064.5625|
| stddev|      null|  null|48.81278008882509|32523.847106081826|2651.901827593075| 48554.51492734062| 1717.175544497582|2761.9159469720103|
|    min|     Ajmer| Appd.|                0|               151|                0|             27151|                78|              2873|
|    max|Trivandrum|

#Data wrangling#




In [14]:
!pip install outlyzer -q


In [15]:
from Outlyzer.zscore import detect_outliers_zscore
from Outlyzer.iqr import detect_outliers_iqr

In [16]:
# convert spark dataframe into pandas dataframe
pandas_df = df.toPandas()
type(pandas_df)

pandas.core.frame.DataFrame

In [27]:
outliers_zscore = detect_outliers_zscore(pandas_df)

#this will show only ture rows
outliers_df = outliers_zscore[outliers_zscore.any(axis=1)]
print(outliers_df)

     CTSA   GOVT  GOVT AIDED  INDEPENDENT    JNV     KV  Region  Status
21  False   True       False        False  False  False   False   False
22  False   True       False        False  False  False   False   False
24  False  False        True        False  False  False   False   False
25  False  False        True        False  False  False   False   False


In [28]:
# Identify columns with non-numeric data
non_numeric_columns = pandas_df.select_dtypes(exclude=[np.number]).columns

# Exclude non-numeric columns from the DataFrame
numeric_df = pandas_df.drop(non_numeric_columns, axis=1)
# Apply the outlier detection function to the modified DataFrame
outliers_iqr = detect_outliers_iqr(numeric_df)

# Print the DataFrame with outliers True with there values
outliers_df = pandas_df[outliers_iqr]
print(outliers_df)

   Region Status  CTSA      GOVT  GOVT AIDED  INDEPENDENT  JNV  KV
0     NaN    NaN   NaN       NaN         NaN       102253  NaN NaN
1     NaN    NaN   NaN       NaN         NaN       101822  NaN NaN
2     NaN    NaN   NaN       NaN         NaN       100178  NaN NaN
3     NaN    NaN   NaN       NaN         NaN        60667  NaN NaN
4     NaN    NaN   NaN       NaN         NaN        60555  NaN NaN
5     NaN    NaN   NaN       NaN         NaN        60089  NaN NaN
6     NaN    NaN   NaN       NaN         NaN        84877  NaN NaN
7     NaN    NaN   NaN       NaN         NaN        84390  NaN NaN
8     NaN    NaN   NaN       NaN         NaN        79048  NaN NaN
9     NaN    NaN   NaN       NaN         NaN        81012  NaN NaN
10    NaN    NaN   NaN       NaN         NaN        80621  NaN NaN
11    NaN    NaN   NaN       NaN         NaN        78070  NaN NaN
12    NaN    NaN   NaN       NaN         NaN       112557  NaN NaN
13    NaN    NaN   NaN       NaN         NaN       112317  NaN

In [55]:
pandas_df.head()

,Region,Status,CTSA,GOVT,GOVT AIDED,INDEPENDENT,JNV,KV
0,Ajmer,Regd.,0,8537,0,102253,4945,9194
1,Ajmer,Appd.,0,8509,0,101822,4928,9160
2,Ajmer,Pass,0,8154,0,100178,4911,8875
3,Bengaluru,Regd.,99,151,0,60667,2442,5108
4,Bengaluru,Appd.,97,151,0,60555,2440,5093


In [54]:
# Create the bar chart
fig = px.bar(pandas_df, x='Region', y='INDEPENDENT', color='Status',
             title='Number of Independent Schools by Region',
             labels={'INDEPENDENT': 'Number of Independent Schools', 'Region': 'Region', 'Status': 'Status'})

# Add hover data labels
fig.update_traces(hovertemplate='Region: %{x}<br>Status: %{customdata[0]}<br>Number of Independent Schools: %{y:,}')

# Customize layout
fig.update_layout(
    xaxis=dict(title='Region'),
    yaxis=dict(title='Number of Independent Schools'),
    legend_title='Status',
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Set plot background color to transparent
    hoverlabel=dict(bgcolor='white', font_size=15),
    hovermode='x'  # Display hover information along the x-axis
)

# Show the chart
fig.show()

In [53]:
# Calculate the percentage for each school type
total_counts = pandas_df[['GOVT', 'GOVT AIDED', 'INDEPENDENT', 'JNV', 'KV']].sum()
percentages = total_counts / total_counts.sum()

# Create labels for the pie chart
labels = ['GOVT', 'GOVT AIDED', 'INDEPENDENT', 'JNV', 'KV']
label_values = [f'{label} ({percent:.1%})' for label, percent in zip(labels, percentages)]

# Define custom colors for the pie chart
colors = px.colors.qualitative.Set2

# Create the pie chart trace
trace = go.Pie(labels=label_values, values=total_counts, hoverinfo='label+percent', textinfo='none',
               marker=dict(colors=colors))

# Define the layout for the pie chart
layout = go.Layout(
    title='Percentage Distribution of School Types',
    showlegend=True,
    legend_title='School Types',
    title_font=dict(size=18, color='black', family='Arial'),
    uniformtext=dict(minsize=12, mode='hide')
)

# Create the figure and add the trace
fig = go.Figure(data=[trace], layout=layout)

# Show the pie chart
fig.show()

In [56]:
# Add an index column to the dataframe
pandas_df['Index'] = np.arange(len(pandas_df))

# Sort the dataframe by Region and GOVT counts
pandas_df_sorted = pandas_df.sort_values(['Region', 'GOVT'])

# Create a line plot to track the change in school counts over time
line_fig = go.Figure()
line_fig.add_trace(go.Scatter(x=pandas_df_sorted['Index'], y=pandas_df_sorted['GOVT'], mode='lines', name='GOVT'))
line_fig.update_layout(title='Change in GOVT School Counts over Time by Region',
                      xaxis=dict(title='Index'), yaxis=dict(title='Number of Schools'))

# Create the box plot
box_fig = px.box(pandas_df, x='Region', y='GOVT', color='Status',
                 title='Distribution of GOVT School Counts by Region and Status',
                 labels={'value': 'Number of Schools', 'Region': 'Region', 'Status': 'Status'})

# Create subplots
fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Change in GOVT School Counts over Time by Region",
                                                                         "Distribution of GOVT School Counts by Region and Status"))
fig.append_trace(line_fig['data'][0], row=1, col=1)
fig.append_trace(box_fig['data'][0], row=2, col=1)
fig.update_layout(height=800)

# Show the combined plot
fig.show()
